<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


### Hands On Lab - Saving and loading a SparkML model


#### Objectives:
In this lab you will
 - Create a simple Linear Regression Model
 - Save the SparkML model
 - Load the SparkML model
 - Make predictions using the loaded SparkML model


#### Install pyspark


In [ ]:
!pip install pyspark
!pip install findspark

#### Import libraries


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

#### Creating the spark session and context


In [ ]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

#### Importing Spark ML libraries


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

#### Create a DataFrame with sample data


In [ ]:
# Create a simple data set of infant height(cms) weight(kgs) chart.

mydata = [[46,2.5],[51,3.4],[54,4.4],[57,5.1],[60,5.6],[61,6.1],[63,6.4]]
  
# Mention column names of dataframe
columns = ["height", "weight"]
  
# creating a dataframe
mydf = spark.createDataFrame(mydata, columns)
  
# show data frame
mydf.show()

#### Converting data frame columns into feature vectors

In this task we use the `VectorAssembler()` function to convert the dataframe columns into feature vectors. 
For our example, we use the horsepower ("hp) and weight of the car as input features and the miles-per-gallon ("mpg") as target labels.


In [ ]:
assembler = VectorAssembler(
    inputCols=["height"],
    outputCol="features")

data = assembler.transform(mydf).select('features','weight')

In [ ]:
data.show()

#### Create and Train model

We can create the model using the `LinearRegression()` class and train using the `fit()` function. 


In [ ]:
# Create a LR model
lr = LinearRegression(featuresCol='features', labelCol='weight', maxIter=100)
lr.setRegParam(0.1)
# Fit the model
lrModel = lr.fit(data)

#### Save the model


In [ ]:
lrModel.write().overwrite().save('infantheight2.model')

#### Load the model


In [ ]:
# You need LinearRegressionModel to load the model
from pyspark.ml.regression import LinearRegressionModel

In [ ]:
model = LinearRegressionModel.load('infantheight2.model')

#### Make Prediction


#### Predict the weight of an infant whose height is 70 CMs.


In [ ]:
# This function converts a scalar number into a dataframe that can be used by the model to predict.
def predict(height):
    assembler = VectorAssembler(inputCols=["height"], outputCol="features")  # Adjusted input column name
    data = [[height, 0]]  # Changed input to reflect height
    columns = ["height", "weight"]  # Updated column names for clarity
    df = spark.createDataFrame(data, columns)
    transformed_df = assembler.transform(df).select('features', 'weight')  # Updated column selection
    predictions = model.transform(transformed_df)
    predictions.select('prediction').show()


In [ ]:
predict(70)

### Practice exercises


#### Save the model as `babyweightprediction.model`


Double-click __here__ for the solution.

<!-- Hint:

lrModel.write().overwrite().save('babyweightprediction.model')

-->


#### Load the model `babyweightprediction.model`


Double-click __here__ for the solution.

<!-- Hint:

model = LinearRegressionModel.load('babyweightprediction.model')
-->


#### Predict the weight of an infant whose height is 50 CMs.


Double-click __here__ for the solution.

<!-- Hint:

predict(50)
-->
